# Anatomy of Redis Web Application

- login cookies
- shopping cart cookies
- caching generated web pages
- caching database rows
- analysing web page visits

## Login Cookies

We can use hash to store the mapping of login user token to the user. 

To set the token:

```
HSET login: token123 user1
ZADD recent: token123 1565576538
```

To get the token:

```
HGET login: token123
```


In [1]:
def check_token(conn, token):
    return conn.hget('login:', token)

In [3]:
def update_token(conn, token, user, item=None):
    timestamp = time.time()
    
    # Keep a mapping from the token to the logged-in user.
    conn.hset('login:', token, user)
    
    # Record when the token was last seen.
    con.zadd('recent:', token, timestamp)
    
    # Record that the user viewed the item.
    if item:
        conn.zadd('viewed:' + token, item, timestamp)
        
        # Remove old items, keeping the most recent 25.
        conn.zremrangebyrank('viewed:' + token, 0, -26)

In [4]:
QUIT = False
LIMIT = 1_000_000

def clean_session(conn):
    while not QUIT:
        size = conn.zcard('recent:')
        if size <= LIMIT:
            time.sleep(1)
            continue
        
        end_index = min(size - LIMIT, 1_000)
        tokens = conn.zrange('recent:', 0, end_index-1)
        
        session_keys = []
        for token in tokens:
            session_keys.append('viewed:' + token)
            
        conn.delete(*session_keys)
        conn.hdel('login:', *tokens)
        conn.zrem('recent:', *tokens)